In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

## 抓取页面信息

In [2]:
start = range(0, 250, 25)
start

range(0, 250, 25)

In [3]:
html = []

In [4]:
def download_html():
    for si in start:
        url = f"https://movie.douban.com/top250?start={si}"
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36"}
        res = requests.get(url, headers=headers)
        if res.status_code != 200:
            raise Exception("error")
        html.append(res.text)

## 解析页面元素

In [5]:
download_html()

In [6]:
def parse_html(html):
    href = []
    title = []
    director = []
    starring = []
    year = []
    country = []
    style = []
    rating_num = []
    evaluate_num = []
    quote = []
    for hi in html:
        bs = BeautifulSoup(hi, "html.parser")
        info = bs.find_all("div", class_="info")
        for infoi in info:
            href.append(infoi.a.attrs['href'])
            title.append(infoi.span.text)
            content = infoi.p.text.strip().split('\n')
#             people = re.search(r'导演: (.*)\xa0\xa0\xa0主演: (.*)', content[0])
#             director.append(people.group(1))
#             starring.append(people.group(2))
            information = re.search(r'(.*)\xa0/\xa0(.*)\xa0/\xa0(.*)', content[1].strip())
            year.append(information.group(1))
            country.append(information.group(2))
            style.append(information.group(3))
            rating_num.append(infoi.find('span', class_='rating_num').text)
            evaluate_num.append(int(infoi.find('div', class_='star').find_all('span')[-1].text[:-3]))
            try:
                quote.append(infoi.find('span', class_='inq').text)
            except:
                quote.append('')
    return [href, title, year, country, style, rating_num, evaluate_num, quote]

## 保存数据

In [7]:
data = parse_html(html)

In [8]:
def save_data(data):
    [href, title, year, country, style, rating_num, evaluate_num, quote] = data
    df = pd.DataFrame()
    df['title'] = title
    df['year'] = year
    df['country'] = country
    df['style'] = style
    df['rating_num'] = rating_num
    df['evaluate_num'] = evaluate_num
    df['quote'] = quote
    df['href'] = href
    return df

In [9]:
df = save_data(data)

In [10]:
df.head()

,title,year,country,style,rating_num,evaluate_num,quote,href
0,肖申克的救赎,1994,美国,犯罪 剧情,9.7,2587809,希望让人自由。,https://movie.douban.com/subject/1292052/
1,霸王别姬,1993,中国大陆 中国香港,剧情 爱情 同性,9.6,1921783,风华绝代。,https://movie.douban.com/subject/1291546/
2,阿甘正传,1994,美国,剧情 爱情,9.5,1944546,一部美国近现代史。,https://movie.douban.com/subject/1292720/
3,泰坦尼克号,1997,美国 墨西哥 澳大利亚 加拿大,剧情 爱情 灾难,9.4,1905861,失去的才是永恒的。,https://movie.douban.com/subject/1292722/
4,这个杀手不太冷,1994,法国 美国,剧情 动作 犯罪,9.4,2099488,怪蜀黍和小萝莉不得不说的故事。,https://movie.douban.com/subject/1295644/


In [11]:
df.to_csv('./top250.csv', encoding='utf_8_sig')